In [1]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-0rLvuRkMiD4Mw25QYygh6rUlZVjpQWNGNF4yez7z3PZ7yCOm",
    # api_key="sk-tMbkq3K1iO5vf0FRMlrmzslGXJZwE0us3mve4QXuvpnZcumG",
    base_url="https://api.chatanywhere.cn/v1"
    # base_url="https://api.chatanywhere.cn/v1"
)

In [2]:
def gpt_35_api(messages: list):

    completion = client.chat.completions.create(model="gpt-3.5-turbo-0125", messages=messages, temperature=0.5)
    return completion.choices[0].message.content

In [3]:
import json
from tqdm import tqdm

In [4]:
from langchain import hub

thought_prompt = hub.pull("tyfann/llm4commit-cot")

In [5]:
gen_prompt = hub.pull("tyfann/llm4commit-cot-gen")

In [6]:
with open(f'../data/angular_filtered/test_data_js.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)
info_list = []
for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
    messages = thought_prompt.invoke(
        {"Question": data['diff']}
    ).to_messages()
    thought = gpt_35_api([{'role': 'user','content': messages[0].content},])
    info_list.append(thought)
    
for item, info in zip(org_data, info_list):
    item['info'] = info

output_file = f'../data/angular_filtered/cot/cot_test_data_info.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data, f, ensure_ascii=False, indent=4)

Processing documents: 100%|██████████| 497/497 [12:00<00:00,  1.45s/it]


In [7]:
gpt_msg = []
for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
    final_messages = gen_prompt.invoke(
        {"Question": data['diff'], "Info": data['info']}
    ).to_messages()
    final_gen = gpt_35_api([{'role': 'user','content': final_messages[0].content},])
    gpt_msg.append(final_gen)

for item, msg in zip(org_data, gpt_msg):
    item['chatgpt_cot'] = msg

output_file = f'../data/angular_filtered/test_result/test_data_js_gpt35_cot.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data, f, ensure_ascii=False, indent=4)

Processing documents: 100%|██████████| 497/497 [07:52<00:00,  1.05it/s]


In [18]:
with open(f'../data/final_preprocessed_data/discriminator/dis_train_data.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)[:100]
gpt_msg = []
for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
    messages = thought_prompt.invoke(
        {"Question": data['diff']}
    ).to_messages()
    thought = gpt_35_api([{'role': 'user','content': messages[0].content},])
    final_messages = gen_prompt.invoke(
        {"Question": data['diff'], "Info": thought}
    ).to_messages()
    final_gen = gpt_35_api([{'role': 'user','content': final_messages[0].content},])
    gpt_msg.append(final_gen)
    
for item, msg in zip(org_data, gpt_msg):
    item['chatgpt_cot'] = msg

output_file = f'../data/final_preprocessed_data/discriminator/dis_train_data_chatgpt_cot_v2.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data, f, ensure_ascii=False, indent=4)

In [13]:
gpt_msg[0]

In [5]:
for lang in ['js']:
    with open(f'../data/final_preprocessed_data/{lang}_baseline_test_data.json', 'r', encoding='UTF-8') as f:
        org_data = json.load(f)[:1000]
    example_q = """"diff --git a/java/client/src/org/openqa/selenium/htmlunit/HtmlUnitAlert.java  b/java/client/src/org/openqa/selenium/htmlunit/HtmlUnitAlert.java \npublic void sendKeys(String keysToSend){ \npublic void authenticateUsing(Credentials credentials){}+@Override+public void setCredentials(Credentials credentials){+}++@Override \npublic void handleAlert(Page page,String message){ \nQueue<String>queue=queues.get(page); \n" """
    example_a = """First, the scope of code changes is to add a method, and second, according to the filename "HtmlUnitAlert", and the function of the file could be an alert, the modification "@Overide" is an annotation which can solve the compilation issue, so the modification can help solve the compilation problem to the alert file. So the commit message for the code diff is "fix compilation issue because of new method in Alert"."""
    gpt_msg = []
    for index, data in tqdm(enumerate(org_data), total=len(org_data), desc="Processing documents"):
        messages = prompt.invoke(
            {"Example_Question": example_q, "Example_Answer": example_a, "Question": data['diff']}
        ).to_messages()
        example_prompt = [{'role': 'user','content': messages[0].content},]
        try:
            gpt_msg.append(gpt_35_api(example_prompt))
        except:
            print(index)
            gpt_msg.append("")
    for item, msg in zip(org_data, gpt_msg):
        item['chatgpt_cot'] = msg
    
    output_file = f'../data/final_preprocessed_data/{lang}_baseline/{lang}_baseline_cot.json'
    with open(output_file, 'w', encoding='UTF-8') as f:
        json.dump(org_data, f, ensure_ascii=False, indent=4)

In [8]:
# with open('../data/chronicle/rag_dev/zeroshot/chronicle_chatgpt_dev.json', 'r', encoding='UTF-8') as f:
#     chronicle_data = json.load(f)

for item, msg in zip(org_data[:100], gpt_msg):
    item['chatgpt_cot'] = msg

output_file = '../data/scenario/multi_diff/multi_diff_cot_django.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data[:100], f, ensure_ascii=False, indent=4)

In [8]:
with open('../data/scenario/multi_diff/scenario_react_multi_diff_chatgpt_zeroshot.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)
    
example_q = """"diff --git a/java/client/src/org/openqa/selenium/htmlunit/HtmlUnitAlert.java  b/java/client/src/org/openqa/selenium/htmlunit/HtmlUnitAlert.java \npublic void sendKeys(String keysToSend){ \npublic void authenticateUsing(Credentials credentials){}+@Override+public void setCredentials(Credentials credentials){+}++@Override \npublic void handleAlert(Page page,String message){ \nQueue<String>queue=queues.get(page); \n" """
example_a = """First, the scope of code changes is to add a method, and second, according to the filename "HtmlUnitAlert", and the function of the file could be an alert, the modification "@Overide" is an annotation which can solve the compilation issue, so the modification can help solve the compilation problem to the alert file. So the commit message for the code diff is "fix compilation issue because of new method in Alert"."""
gpt_msg = []
for data in tqdm(org_data, total=len(org_data), desc="Processing documents"):
    messages = prompt.invoke(
        {"Example_Question": example_q, "Example_Answer": example_a, "Question": data['diffs']}
    ).to_messages()
    example_prompt = [{'role': 'user','content': messages[0].content},]
    gpt_msg.append(gpt_35_api(example_prompt))

# with open('../data/chronicle/rag_dev/zeroshot/chronicle_chatgpt_dev.json', 'r', encoding='UTF-8') as f:
#     chronicle_data = json.load(f)

for item, msg in zip(org_data, gpt_msg):
    item['chatgpt_cot'] = msg

output_file = '../data/scenario/multi_diff/multi_diff_cot_js.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data, f, ensure_ascii=False, indent=4)

In [16]:
import re

def preprocess(text):
    if re.search(r'commit\s*message', text, re.IGNORECASE):
        pattern = r'\"([^"]+)'
        match = re.search(pattern, text)
        if match:
            return match.group(1)
        else:
            return text
    else:
        return text

In [17]:
with open('../data/msg_nngen_nmt_codebert_chatgpt.json', 'r', encoding='UTF-8') as f:
    org_data = json.load(f)

for item, msg in zip(org_data[:100], gpt_msg):
    item['chatgpt_cot'] = preprocess(msg)

output_file = '../data/chatgpt_cot.json'
with open(output_file, 'w', encoding='UTF-8') as f:
    json.dump(org_data[:100], f, ensure_ascii=False, indent=4)

In [28]:
max_len = 0
for msg in gpt_msg:
    max_len = max(max_len, len(msg.split()))